# [모의 캐글 - 게임] 비매너 댓글 식별 

- 자연어 multi label classification 과제
- 작성자 : MNC Sukyung Kim (skkim@mnc.ai)

참고 논문 : 
- [BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)

# 1. 환경 설정 및 라이브러리 불러오기

In [1]:
# !pip install -r requirements.txt

In [51]:
import pandas as pd
import os
import json
import numpy as np
import shutil

from sklearn.metrics import f1_score
from datetime import datetime, timezone, timedelta
import random
from tqdm import tqdm


from attrdict import AttrDict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils import *
from torch.optim import Adam, AdamW

from transformers import logging, get_linear_schedule_with_warmup


from transformers import ( 
    BertConfig,
    ElectraConfig
)

### v2 에서 라이브러리 추가됨
# 실험에 사용하실 모델 라이브러리를 추가하시는 걸 잊지 마세요!

from transformers import (
    BertTokenizer,  
    AutoTokenizer,
    ElectraTokenizer,
    AlbertTokenizer,
    
)

from transformers import (
    BertModel,
    AutoModel, 
    ElectraForSequenceClassification,
    BertForSequenceClassification,
    AlbertForSequenceClassification,
    AutoModelForSequenceClassification
)


In [52]:
# 사용할 GPU 지정
print("number of GPUs: ", torch.cuda.device_count())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = torch.cuda.is_available()
print("Does GPU exist? : ", use_cuda)
DEVICE = torch.device("cuda" if use_cuda else "cpu")

number of GPUs:  1
Does GPU exist? :  True


In [53]:
# True 일 때 코드를 실행하면 example 등을 보여줌
DEBUG = True

In [54]:
# config 파일 불러오기
config_path = os.path.join('config_electra_v3_normalized.json')

def set_config(config_path):
    if os.path.lexists(config_path):
        with open(config_path) as f:
            args = AttrDict(json.load(f))
            print("config file loaded.")
            print(args.pretrained_model)
    else:
        assert False, 'config json file cannot be found.. please check the path again.'
    
    return args


# 코드 중간중간에 끼워넣어 리셋 가능
args = set_config(config_path)

# 결과 저장 폴더 미리 생성
# os.makedirs(args.result_dir, exist_ok=True)
# os.makedirs(args.config_dir, exist_ok=True)

config file loaded.
monologg/koelectra-base-v3-hate-speech


# 2. EDA 및 데이터 전처리

In [56]:
# data 경로 설정  
train_path = os.path.join(args.data_dir,'train_normalized2.csv')

print("train 데이터 경로가 올바른가요? : ", os.path.lexists(train_path))


train 데이터 경로가 올바른가요? :  True


### 2-1. Train 데이터 확인

In [57]:
train_df = pd.read_csv(train_path,encoding = 'UTF-8-SIG') # encoding = 'UTF-8-SIG' 원본

train_df.head()

,title,comment,bias,hate
0,미스터 션샤인 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해 진짜,none,none
1,극사실주의 현실가장 보통의 연애 김래원X공효진 16년만의 랑데부,공효진 발 연기나 이 질 생각이 없던데 왜 계속 주연일까,none,hate
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,others,hate
3,섹션TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none
4,임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창 됐으면,난 절대로 임현주 욕하는 인간이랑은 안 논다,none,none


In [58]:
train_df = train_df.astype('str')

In [59]:
train_df.drop([2650,3780,6310],inplace=True)
train_df.reset_index(inplace=True,drop=True)

In [61]:
train_df

,title,comment,bias,hate
0,미스터 션샤인 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해 진짜,none,none
1,극사실주의 현실가장 보통의 연애 김래원X공효진 16년만의 랑데부,공효진 발 연기나 이 질 생각이 없던데 왜 계속 주연일까,none,hate
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,others,hate
3,섹션TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none
4,임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창 됐으면,난 절대로 임현주 욕하는 인간이랑은 안 논다,none,none
...,...,...,...,...
8359,배우 이필립 SNS 스타 연인에게 초호화 프러포즈 눈길,아니 근데 튜닝 한 사람은 프러 포즈 받지도 결혼도 못함 ᄏᄏᄏ 자기들은 돈 없어서...,others,hate
8360,마약백스텝김새롬 탓 실형 피한 이찬오 이미지는 치명상,그러니깐 여자를 잘 만나야 돼 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯...,gender,hate
8361,그들만의 세상홍상수김민희 새해데이트에 반응싸늘,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다,none,none
8362,시크릿 마더 김소연 누가 죽였나송윤아와 갈등,재미가 없어요,none,none


In [10]:
### v2 에서 추가됨

# title 중 가장 긴 타이틀 길이
max_len_title = np.max(train_df['title'].str.len())
max_len_title

45

In [11]:
# comment 중 가장 긴 타이틀 길이
max_len_comment=np.max(train_df['comment'].str.len())
max_len_comment

137

In [12]:
# 길이가 128이 넘는 코멘트 확인
train_df['comment'][train_df['comment'].str.len()>128]

136     진짜 연기 못하는 애들만 모아뒀네 월화 드라마가 시청률 똥 마이라 드라마 잠정 폐지...
170     개 쳐늙으면 꼴 보기 싫어지는 게 사람의 마음이 지 얘네야 광고 수익으로 번 게 많...
256     남자 지갑 꺼내 어쩌 내 말 많은데 쟤 갤러리아 포레사는 데 한국에서 10위권 안에...
424     쿵쾅이 들아 너네가 알아야 할 게 눈코 입 윤곽 다 해도 겨우 중 타치든가 다 해도...
657     수아가 불쌍하다 납치돼서 20년을 힘들게 살고 남자 놈은 수아 버리고 민 채린한테 ...
1236    예능에서 자주 나오는 웃기려고 하는 거지만 상당히 정확한 전기 오는 거짓말탐지기 유...
1296    어쩐지 1위부터 와 꾸가 빻 고 나머지들 와 꾸가 개 빻았길래 이 게 아이돌이야 싶...
1335    송민호 혼자 왜 설거지 시켰어요 골병들겠다 피오가 많이 도와줘야 할 텐데 송민호 키...
1437    그냥 좀 착하게 살면 안 되겠냐 여기다가 설정이 네 어쩌 네 썰 전 펼치면서 애 하...
1609    역시 힙합은 마약과 떨어질 수 없는 자르지 그만큼 한국 오버 힙합은 YG가 책임지고...
1643    세 가족 모두 짠했지만 반듯하게 잘 크고 있는 건 아빠의 빈자리를 엄마의 역할이 배...
1929    두 분 가정살이에 왜 남들이 이래저래 하는지 그런 면 두 분이 일반이면 저렇게까지 ...
2066    솔직히 남자들 사이에서 저런 거 올리면 욕하기도 그렇고 좀 가만있게 될 거 같다 로...
2567    솔직히 한 혜진 이제 나이가 37살 되는데 사실 일반인 여자이면 결혼 이미 포기하는...
2654    내가 너를 고소하고 싶어 정국 아 진짜 존 나 패고 싶다 생각이 없니 국뽕으로 뭘 ...
2701    사장들 긴장해서 인터뷰 장소에서 지적당하는 화면 보며 더 우울해하는데 옆에 조보아 ...
3067    솔까 마다 욕하는 인간들 치고 돈 안 떼인 인간 없지 싶다 안 그러고는 불구대천 원...
3264    댓글들에 역

In [13]:
len(train_df)

8364

In [14]:
train_df

,title,comment,bias,hate
0,미스터 션샤인 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해 진짜,none,none
1,극사실주의 현실가장 보통의 연애 김래원X공효진 16년만의 랑데부,공효진 발 연기나 이질 생각이 없던데 왜 계속 주연일까,none,hate
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,others,hate
3,섹션TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none
4,임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창 됐으면,난 절대로 임현주 욕하는 인간이랑은 안 논다,none,none
...,...,...,...,...
8359,배우 이필립 SNS 스타 연인에게 초호화 프러포즈 눈길,아니 근데 튜닝한 사람은 프러 포즈 받지도 결혼도 못함 ᄏᄏᄏ 자기들은 돈 없어서 ...,others,hate
8360,마약백스텝김새롬 탓 실형 피한 이찬오 이미지는 치명상,그러니깐 여자를 잘 만나야 돼 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯...,gender,hate
8361,그들만의 세상홍상수김민희 새해데이트에 반응싸늘,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다,none,none
8362,시크릿 마더 김소연 누가 죽였나송윤아와 갈등,재미가 없어요,none,none


In [15]:
# print("bias classes: ", train_df.bias.unique())
# print("hate classes: ", train_df.hate.unique())

bias classes:  ['none' 'others' 'gender']
hate classes:  ['none' 'hate']


In [16]:
# pd.crosstab(train_df.bias, train_df.hate, margins=True)

hate,hate,none,All
bias,,,
gender,1216,83,1299
none,2065,3422,5487
others,1437,141,1578
All,4718,3646,8364


### 2-2. Test 데이터 확인

In [62]:
test_path = os.path.join(args.data_dir,'test_normalized2.csv')
print("test 데이터 경로가 올바른가요? : ", os.path.lexists(test_path))

test 데이터 경로가 올바른가요? :  True


In [63]:
test_df = pd.read_csv(test_path)
test_df = test_df.astype('str')
test_df.head()

,ID,title,comment
0,0,류현경박성훈 공개연애 4년차 애정전선 이상無의지 많이 된다,둘 다 너무 좋다 행복하세요
1,1,현금 유도1인 1라면골목식당 백종원 초심 잃은 도시락집에 경악,근데 만 원 이하는 현금 결제만 하라고 써놓은 집 우리나라에 엄청 많은데
2,2,입대 D11 서은광의 슬픈 멜로디비투비 눈물의 첫 체조경기장,누군데 얘네
3,3,아이콘택트 리쌍 길 3년 전 결혼설 부인한 이유 공개결혼출산 숨겼다,쇼하지 마라 자식 아 음주 1번은 실수 2번은 고의 3번은 인간쓰레기 다 슬 금슬은...
4,4,구하라 안검하수 반박 해프닝당당하다vs그렇게까지 설전,안검하수 가지고 있는 분께 희망을 주고 싶은 건가요 수술하면 이렇게 자연스러워진다고...


In [175]:
len(test_df)

511

### 2-3. 데이터 전처리 (Label Encoding)
bias, hate 라벨들의 class를 정수로 변경하여 라벨 인코딩을 하기 위한 딕셔너리입니다.

- bias, hate 컬럼을 합쳐서 하나의 라벨로 만들기 

In [20]:
# 두 라벨의 가능한 모든 조합 만들기
# combinations = np.array(np.meshgrid(train_df.bias.unique(), train_df.hate.unique())).T.reshape(-1,2)

# if DEBUG==True:
#     print(combinations)

[['none' 'none']
 ['none' 'hate']
 ['others' 'none']
 ['others' 'hate']
 ['gender' 'none']
 ['gender' 'hate']]


In [21]:
# bias, hate 컬럼을 합친 것
# bias_hate = list(np.array([train_df['bias'].values, train_df['hate'].values]).T.reshape(-1,2))

# if DEBUG==True:
#     print(bias_hate[:5])


[array(['none', 'none'], dtype=object), array(['none', 'hate'], dtype=object), array(['others', 'hate'], dtype=object), array(['none', 'none'], dtype=object), array(['none', 'none'], dtype=object)]


In [22]:
# labels = []
# for i, arr in enumerate(bias_hate):
#     for idx, elem in enumerate(combinations):
#         if np.array_equal(elem, arr):
#             labels.append(idx)

# train_df['label'] = labels
# train_df.head()

,title,comment,bias,hate,label
0,미스터 션샤인 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해 진짜,none,none,0
1,극사실주의 현실가장 보통의 연애 김래원X공효진 16년만의 랑데부,공효진 발 연기나 이질 생각이 없던데 왜 계속 주연일까,none,hate,1
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,others,hate,3
3,섹션TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,none,none,0
4,임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창 됐으면,난 절대로 임현주 욕하는 인간이랑은 안 논다,none,none,0


In [64]:
# hate speech 모델도 아웃풋이 3개로 나오는 모델이라 따로 라벨을 만들어줌 원래 offensive 항목이 있었으나 이자료에서는 hate로 통합했기때문에 그냥 씀
train_df['bias'].replace({"none":0, "gender":1, "others":2}, inplace=True)
train_df['hate'].replace({"none":0, "hate":1}, inplace=True)
train_df.head()

,title,comment,bias,hate
0,미스터 션샤인 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해 진짜,0,0
1,극사실주의 현실가장 보통의 연애 김래원X공효진 16년만의 랑데부,공효진 발 연기나 이 질 생각이 없던데 왜 계속 주연일까,0,1
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,2,1
3,섹션TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,0,0
4,임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창 됐으면,난 절대로 임현주 욕하는 인간이랑은 안 논다,0,0


In [132]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8364 entries, 0 to 8363
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    8364 non-null   object
 1   comment  8364 non-null   object
 2   bias     8364 non-null   int64 
 3   hate     8364 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 261.5+ KB


## 3. Dataset 로드

### 3-0. Pre-trained tokenizer 탐색

In [65]:
# config.json 에서 지정 이름별로 가져올 라이브러리 지정

TOKENIZER_CLASSES = {
    "BertTokenizer": BertTokenizer,
    "AutoTokenizer": AutoTokenizer,
    "ElectraTokenizer": ElectraTokenizer,
    "AlbertTokenizer": AlbertTokenizer,
    }


- Tokenizer 사용 예시

In [66]:
TOKENIZER = TOKENIZER_CLASSES[args.tokenizer_class].from_pretrained(args.pretrained_model)
if DEBUG==True:
    print(TOKENIZER)

PreTrainedTokenizerFast(name_or_path='monologg/koelectra-base-v3-hate-speech', vocab_size=35000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


# [모의 캐글 - 게임] 비매너 댓글 식별 

- 자연어 multi label classification 과제
- 작성자 : MNC Sukyung Kim (skkim@mnc.ai)

참고 논문 : 
- [BERT: Pre-training of Deep Bidirectional Transformers for
Language Understanding](https://arxiv.org/pdf/1810.04805.pdf)
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)

In [52]:
train_df['title'][0]

'미스터 션샤인 변요한 김태리와 같은 양복 입고 학당 방문 이유는'

In [53]:
train_df['comment'][0]


'김태리 정말 연기 잘해 진짜'

In [28]:
if DEBUG == True:
    example = train_df['title'][0]
    comment_ex = train_df['comment'][0]
    print(TOKENIZER(comment_ex))

{'input_ids': [2, 9945, 4108, 6595, 7144, 3258, 4151, 7082, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [54]:
if DEBUG==True:
    print(TOKENIZER.encode(example),"\n")
    
    # 토큰으로 나누기
    print(TOKENIZER.tokenize(example),"\n")
    
    print(TOKENIZER.tokenize(comment_ex),"\n")
    
    # 토큰 id로 매핑하기
    print(TOKENIZER.convert_tokens_to_ids(TOKENIZER.tokenize(example)))


[2, 11100, 2936, 4825, 4139, 2772, 4150, 4283, 9945, 4108, 4192, 2024, 4112, 18723, 3247, 4219, 31950, 6673, 6372, 4034, 3] 

['미스터', '션', '##샤', '##인', '변', '##요', '##한', '김태', '##리', '##와', '같', '##은', '양복', '입', '##고', '학당', '방문', '이유', '##는'] 

['김태', '##리', '정말', '연기', '잘', '##해', '진짜'] 

[11100, 2936, 4825, 4139, 2772, 4150, 4283, 9945, 4108, 4192, 2024, 4112, 18723, 3247, 4219, 31950, 6673, 6372, 4034]


### 3-1. Dataset 만드는 함수 정의

In [67]:
train_df.drop(['bias'],axis=1,inplace=True) # bias 라벨을 떨궈줌 hate 라벨만 쓰기위해

In [68]:
train_df.columns = ['title','comment','label']

In [69]:
train_df

,title,comment,label
0,미스터 션샤인 변요한 김태리와 같은 양복 입고 학당 방문 이유는,김태리 정말 연기 잘해 진짜,0
1,극사실주의 현실가장 보통의 연애 김래원X공효진 16년만의 랑데부,공효진 발 연기나 이 질 생각이 없던데 왜 계속 주연일까,1
2,손연재 리듬체조 학원 선생님 하고 싶은 일 해서 행복하다,누구처럼 돈만 밝히는 저급 인생은 살아가지 마시길 행복은 머니 순이 아니니깐 작은 ...,1
3,섹션TV 김해숙 허스토리 촬영 후 우울증 얻었다,일본 축구 져라,0
4,임현주 아나운서 노브라 챌린지 방송 덕에 낸 용기 자연스런 논의의 창 됐으면,난 절대로 임현주 욕하는 인간이랑은 안 논다,0
...,...,...,...
8359,배우 이필립 SNS 스타 연인에게 초호화 프러포즈 눈길,아니 근데 튜닝 한 사람은 프러 포즈 받지도 결혼도 못함 ᄏᄏᄏ 자기들은 돈 없어서...,1
8360,마약백스텝김새롬 탓 실형 피한 이찬오 이미지는 치명상,그러니깐 여자를 잘 만나야 돼 징글징글한 것들 만나면 인생 끝가지 돌아가게 되는 듯...,1
8361,그들만의 세상홍상수김민희 새해데이트에 반응싸늘,참으로 아름다운 커플입니다 늘 행복하시고 새해에도 늘 꽃길만 걸으시길 축원합니다,0
8362,시크릿 마더 김소연 누가 죽였나송윤아와 갈등,재미가 없어요,0


In [75]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len, mode = 'train'):

        self.data = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.mode = mode
        
        if self.mode!='test':
            try: 
                self.labels = df['label'].tolist()
            except:
                assert False, 'CustomDataset Error : \'label\' column does not exist in the dataframe'
     
    def __len__(self):
        return len(self.data)
                

    def __getitem__(self, idx):
        """
        전체 데이터에서 특정 인덱스 (idx)에 해당하는 기사제목과 댓글 내용을 
        토크나이즈한 data('input_ids', 'attention_mask','token_type_ids')의 딕셔너리 형태로 불러옴
        """
        title = self.data.title.iloc[idx]
        comment = self.data.comment.iloc[idx]
        
        tokenized_text = self.tokenizer(comment, #title 은 학습에 사용하지 않아 지웠음.
                             padding= 'max_length',
                             max_length=self.max_len,
                             truncation=True,
                             return_token_type_ids=True,
                             return_attention_mask=True,
                             return_tensors = "pt")
        
        data = {'input_ids': tokenized_text['input_ids'].clone().detach().long(),
               'attention_mask': tokenized_text['attention_mask'].clone().detach().long(),
               'token_type_ids': tokenized_text['token_type_ids'].clone().detach().long(),
               }
        
        if self.mode != 'test':
            label = self.data.label.iloc[idx]
            return data, label
        else:
            return data
        

    
train_dataset = CustomDataset(train_df, TOKENIZER,args.max_seq_len , mode ='train')
print("train dataset loaded.")

train dataset loaded.


In [76]:
if DEBUG ==True :
    print("dataset sample : ")
    print(train_dataset[1])

dataset sample : 
({'input_ids': tensor([[    2,  2085, 30497,  2735,  7144,  4065,  3240,  3351,  6243,  4007,
          3123,  4820,  4244,  3178,  6437, 12242,  4366,  4149,     3,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,

In [20]:
# encoded_plus = tokenizer.encode_plus(
#                     sentence,                      # Sentence to encode.
#                     add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                     max_length = 128,           # Pad & truncate all sentences.
#                     pad_to_max_length = True,
#                     return_attention_mask = True,   # Construct attention masks.
#                     return_tensors = 'pt',     # Return pytorch tensors.
#                )

### 3-2. Train, Validation set 나누기

In [77]:
from sklearn.model_selection import train_test_split
                                                         
train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=args.seed)

train_dataset = CustomDataset(train_data, TOKENIZER, args.max_seq_len, 'train')
val_dataset = CustomDataset(val_data, TOKENIZER, args.max_seq_len, 'validation')

print("Train dataset: ", len(train_dataset))
print("Validation dataset: ", len(val_dataset))

Train dataset:  7527
Validation dataset:  837


In [22]:
train_dataset[0][0]['input_ids'][0]

tensor([   2, 3308, 4113, 4007, 2010, 2245, 4050, 4140, 6683, 4007, 6327, 6231,
        4176, 6226, 4112,    3,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])

In [23]:
len(val_dataset[0][0]['input_ids'][0])

80

## 4. 분류 모델 학습을 위한 세팅

### 4-1. 아키텍쳐 설정




- [PretrainedConfig](https://huggingface.co/docs/transformers/v4.16.2/en/main_classes/configuration#transformers.PretrainedConfig.from_pretrained)
-[KcELECTRA 사전학습 모델](https://github.com/Beomi/KcELECTRA)

In [78]:
from transformers import logging    
logging.set_verbosity_error()

# config.json 에 입력된 architecture 에 따라 베이스 모델 설정
BASE_MODELS = {
    "BertForSequenceClassification": BertForSequenceClassification,
    "AutoModel": AutoModel,
    "ElectraForSequenceClassification": ElectraForSequenceClassification,
    "AlbertForSequenceClassification": AlbertForSequenceClassification,
    "AutoModelForSequenceClassification":AutoModelForSequenceClassification

}


myModel = BASE_MODELS[args.architecture].from_pretrained(args.pretrained_model, 
                                                         num_labels = args.num_classes, 
                                                         output_attentions = False, # Whether the model returns attentions weights.
                                                         output_hidden_states = True # Whether the model returns all hidden-states.
                                                        )
if DEBUG==True:
    # 모델 구조 확인
    print(myModel)   

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [25]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

### 4-2. 모델 설정


- BertForSequenceClassifier (line 1232부터 참고) [source code](https://github.com/huggingface/transformers/blob/a39dfe4fb122c11be98a563fb8ca43b322e01036/src/transformers/modeling_bert.py#L1284-L1287)

- ElectraForSequenceClassifier [source code](https://huggingface.co/transformers/v3.0.2/_modules/transformers/modeling_electra.html#ElectraForSequenceClassification)

- SequenceClassier output 형태 : tuple(torch.FloatTensor)
    - loss (torch.FloatTensor of shape (1,), optional, returned when label is provided):
    Classification (or regression if config.num_labels==1) loss.

   - logits (torch.FloatTensor of shape (batch_size, config.num_labels)):
    Classification (or regression if config.num_labels==1) scores (before SoftMax).

    - hidden_states (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True):
    Tuple of torch.FloatTensor (one for the output of the embeddings + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).

    - Hidden-states of the model at the output of each layer plus the initial embedding outputs.

    - attentions (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True):
Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).

    Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.
    
    
- v2 에서 수정 및 추가 된 부분 많음



In [79]:
### v2 에서 일부 수정됨
class myClassifier(nn.Module):
    def __init__(self, model, hidden_size = 768, num_classes=args.num_classes, selected_layers=False, params=None):
        super(myClassifier, self).__init__()
        self.model = model
        self.softmax = nn.Softmax(dim=1) 
        self.selected_layers = selected_layers
        
        # 사실 dr rate은 model config 에서 hidden_dropout_prob로 가져와야 하는데 bert에선 0.1이 쓰였음
        self.dropout = nn.Dropout(0.1)


    def forward(self, token_ids, attention_mask, segment_ids):      
        outputs = self.model(input_ids = token_ids, 
                             token_type_ids = segment_ids.long(), 
                             attention_mask = attention_mask.float().to(token_ids.device))
        
        # hidden state에서 마지막 4개 레이어를 뽑아 합쳐 새로운 pooled output 을 만드는 시도
        if self.selected_layers == True:
            hidden_states = outputs.hidden_states
            pooled_output = torch.cat(tuple([hidden_states[i] for i in [-4, -3, -2, -1]]), dim=-1)
            # print("concatenated output shape: ", pooled_output.shape)
            ## dim(batch_size, max_seq_len, hidden_dim) 에서 가운데를 0이라 지정함으로, [cls] 토큰의 임베딩을 가져온다. 
            ## (text classification 구조 참고)
            pooled_output = pooled_output[:, 0, :]
            # print(pooled_output)

            pooled_output = self.dropout(pooled_output)

            ## 3개의 레이어를 합치므로 classifier의 차원은 (hidden_dim, 6)이다
            classifier = nn.Linear(pooled_output.shape[1], args.num_classes).to(token_ids.device)
            logits = classifier(pooled_output)
        
        else:
            logits=outputs.logits
        
    
        # 각 클래스별 확률
        prob= self.softmax(logits)
        # print(prob)
        # logits2 = outputs.logits
        # print(self.softmax(logits2))


        return logits, prob
        
# 마지막 4 hidden layers concat 하는 방법을 쓰신다면 True로 변경        
model = myClassifier(myModel, selected_layers=False)

# if DEBUG ==True :
#     print(model)

### 4-3. 모델 구성 확인

In [80]:
if DEBUG==True:
    params = list(model.named_parameters())

    print('The BERT model has {:} different named parameters.\n'.format(len(params)))

    print('==== Embedding Layer ====\n')

    for p in params[0:5]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== First Transformer ====\n')

    for p in params[5:21]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

    print('\n==== Output Layer ====\n')

    for p in params[-4:]:
        print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

model.electra.embeddings.word_embeddings.weight         (35000, 768)
model.electra.embeddings.position_embeddings.weight       (512, 768)
model.electra.embeddings.token_type_embeddings.weight       (2, 768)
model.electra.embeddings.LayerNorm.weight                     (768,)
model.electra.embeddings.LayerNorm.bias                       (768,)

==== First Transformer ====

model.electra.encoder.layer.0.attention.self.query.weight   (768, 768)
model.electra.encoder.layer.0.attention.self.query.bias       (768,)
model.electra.encoder.layer.0.attention.self.key.weight   (768, 768)
model.electra.encoder.layer.0.attention.self.key.bias         (768,)
model.electra.encoder.layer.0.attention.self.value.weight   (768, 768)
model.electra.encoder.layer.0.attention.self.value.bias       (768,)
model.electra.encoder.layer.0.attention.output.dense.weight   (768, 768)
model.electra.encoder.layer.0.attention.output.dense.bi

## 5. 학습 진행

### 5-0. Early Stopper 함수 정의

- v2에서 코드 일부 삭제

In [81]:
class LossEarlyStopper():
    """Early stopper

        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        """ 초기화

        Args:
            patience (int): loss가 줄어들지 않아도 학습할 epoch 수
            weight_path (str): weight 저장경로
            verbose (bool): 로그 출력 여부, True 일 때 로그 출력
        """
        self.patience = patience
        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False

    def check_early_stopping(self, loss: float)-> None:
        # 첫 에폭
        if self.min_loss == np.Inf:
            self.min_loss = loss
           
        # loss가 줄지 않는다면 -> patience_counter 1 증가
        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            # patience 만큼 loss가 줄지 않았다면 학습을 중단합니다.
            if self.patience_counter == self.patience:
                self.stop = True
            print(msg)
        # loss가 줄어듬 -> min_loss 갱신, patience_counter 초기화
        elif loss <= self.min_loss:
            self.patience_counter = 0
            ### v2 에서 수정됨
            ### self.save_model = True -> 삭제 (사용하지 않음)
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss

            print(msg)

### 5-1. Epoch 별 학습 및 검증

- [Transformers optimization documentation](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules)
- [스케줄러 documentation](https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#schedules)
- Adam optimizer의 epsilon 파라미터 eps = 1e-8 는 "계산 중 0으로 나눔을 방지 하기 위한 아주 작은 숫자 " 입니다. ([출처](https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/))
- 스케줄러 파라미터
    - `warmup_ratio` : 
      - 학습이 진행되면서 학습률을 그 상황에 맞게 가변적으로 적당하게 변경되게 하기 위해 Scheduler를 사용합니다.
      - 처음 학습률(Learning rate)를 warm up하기 위한 비율을 설정하는 warmup_ratio을 설정합니다.
  

In [82]:
import wandb

wandb.init(project="week3_nlp_roy", entity="roywoo")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: roywoo (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-03-03 07:45:09.988743: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [83]:
args = set_config(config_path)

logging.set_verbosity_warning()

# 재현을 위해 모든 곳의 시드 고정
seed_val = args.seed
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def train(model, train_data, val_data, args, mode = 'train'):
    
    # args.run은 실험 이름 (어디까지나 팀원들간의 버전 관리 및 공유 편의를 위한 것으로, 자유롭게 수정 가능합니다.)
    print("RUN : ", args.run)
    shutil.copyfile("config.json", os.path.join(args.config_dir, f"config_{args.run}.json"))

    early_stopper = LossEarlyStopper(patience=args.patience)
    
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=args.train_batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=args.train_batch_size)

    
    if DEBUG == True:
        # 데이터로더가 성공적으로 로드 되었는지 확인
        for idx, data in enumerate(train_dataloader):
            if idx==0:
                print("batch size : ", len(data[0]['input_ids']))
                print("The first batch looks like ..\n", data[0])
    
    
    criterion = nn.CrossEntropyLoss()
    
    total_steps = len(train_dataloader) * args.train_epochs

    ### v2에서 수정됨 (Adam -> AdamW)
    optimizer = AdamW(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * args.warmup_proportion), 
                                                num_training_steps=total_steps)

    
    if use_cuda:
        model = model.to(DEVICE)
        criterion = criterion.to(DEVICE)
        

    tr_loss = 0.0
    val_loss = 0.0
    best_score = 0.0
    best_loss= np.inf
      

    for epoch_num in range(args.train_epochs):

            total_acc_train = 0
            total_loss_train = 0
            
            assert mode in ['train', 'val'], 'your mode should be either \'train\' or \'val\''
            
            if mode =='train':
                for train_input, train_label in tqdm(train_dataloader):
                    
                    
                    mask = train_input['attention_mask'].to(DEVICE)
                    input_id = train_input['input_ids'].squeeze(1).to(DEVICE)
                    segment_ids = train_input['token_type_ids'].squeeze(1).to(DEVICE)
                    train_label = train_label.long().to(DEVICE)  
                    
                    ### v2에 수정됨
                    optimizer.zero_grad()
 
                    output = model(input_id, mask, segment_ids)
                    batch_loss = criterion(output[0].view(-1,3), train_label.view(-1)) #예측해야하는 아웃풋 라벨이 6개에서 3개로 줄어 veiw(-1,6)>view(-1,3)으로 바꿔줌
                    total_loss_train += batch_loss.item()

                    acc = (output[0].argmax(dim=1) == train_label).sum().item()
                    total_acc_train += acc
                    
                    ### v2에 수정됨
                    optimizer.zero_grad()
                    
                    batch_loss.backward()
                    optimizer.step()
                    
                    ### v2 에 수정됨
                    scheduler.step()
                    

            total_acc_val = 0
            total_loss_val = 0
            wandb.log({"train_batch_loss": batch_loss,
                      "total_loss_train":total_loss_train,
                      "total_acc_train": total_acc_train,
                      })


            
            # validation을 위해 이걸 넣으면 이 evaluation 프로세스 중엔 dropout 레이어가 다르가 동작한다.
            model.eval()
            
            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    mask = val_input['attention_mask'].to(DEVICE)
                    input_id = val_input['input_ids'].squeeze(1).to(DEVICE)
                    segment_ids = val_input['token_type_ids'].squeeze(1).to(DEVICE)
                    val_label = val_label.long().to(DEVICE)

                    output = model(input_id, mask, segment_ids)
                    ### v2 에서 일부 수정 (output -> output[0]로 myClassifier 모델에 정의된대로 logits 가져옴)
                    batch_loss = criterion(output[0].view(-1,3), val_label.view(-1)) #예측해야하는 아웃풋 라벨이 6개에서 3개로 줄어 veiw(-1,6)>view(-1,3)으로 바꿔줌
                    total_loss_val += batch_loss.item()
                    
                    ### v2 에서 일부 수정 (output -> output[0]로 myClassifier 모델에 정의된대로 logits 가져옴)
                    acc = (output[0].argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            
            train_loss = total_loss_train / len(train_data)
            train_accuracy = total_acc_train / len(train_data)
            val_loss = total_loss_val / len(val_data)
            val_accuracy = total_acc_val / len(val_data)
            
            wandb.log({"val_batch_loss": batch_loss,
                      "total_loss_val":total_loss_val,
                      "total_acc_val": total_acc_val,
                      })            
            # 한 Epoch 학습 후 학습/검증에 대해 loss와 평가지표 (여기서는 accuracy로 임의로 설정) 출력
            print(
                f'Epoch: {epoch_num + 1} \
                | Train Loss: {train_loss: .3f} \
                | Train Accuracy: {train_accuracy: .3f} \
                | Val Loss: {val_loss: .3f} \
                | Val Accuracy: {val_accuracy: .3f}')
          
            # early_stopping check
            early_stopper.check_early_stopping(loss=val_loss)

            if early_stopper.stop:
                print('Early stopped, Best score : ', best_score)
                break

            ### v2 에 수정됨
            ### loss와 accuracy가 꼭 correlate하진 않습니다.
            ### 
            ### 원본 (필요하다면 다시 해제 후 사용)
            # if val_accuracy > best_score : 
            if val_loss < best_loss :
            # 모델이 개선됨 -> 검증 점수와 베스트 loss, weight 갱신
                best_score = val_accuracy 
                
                ### v2에서 추가
                best_loss =val_loss
                # 학습된 모델을 저장할 디렉토리 및 모델 이름 지정
                SAVED_MODEL =  os.path.join(args.result_dir, f'best_{args.run}.pt')
            
                check_point = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'scheduler': scheduler.state_dict()
                }
                torch.save(check_point, SAVED_MODEL)  
              
            # print("scheduler : ", scheduler.state_dict())


    print("train finished")


train(model, train_dataset, val_dataset, args, mode = 'train')
wandb.finish()

config file loaded.
monologg/koelectra-base-v3-hate-speech
RUN :  electra_v3_hate_speech_comment_p0
batch size :  32
The first batch looks like ..
 {'input_ids': tensor([[[    2, 11139,  6470,  ...,     0,     0,     0]],

        [[    2,  3223,  4114,  ...,     0,     0,     0]],

        [[    2, 15967,   284,  ...,     0,     0,     0]],

        ...,

        [[    2,  3080,  4711,  ...,     0,     0,     0]],

        [[    2,  6997,  4880,  ...,     0,     0,     0]],

        [[    2, 12104,  4257,  ...,     0,     0,     0]]]), 'attention_mask': tensor([[[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]]), 'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..

100% 236/236 [01:44<00:00,  2.25it/s]


Epoch: 1                 | Train Loss:  0.016                 | Train Accuracy:  0.795                 | Val Loss:  0.010                 | Val Accuracy:  0.876


100% 236/236 [01:47<00:00,  2.20it/s]


Epoch: 2                 | Train Loss:  0.009                 | Train Accuracy:  0.887                 | Val Loss:  0.010                 | Val Accuracy:  0.875
Validation loss decreased 0.009859162775561515 -> 0.009553241117311066


100% 236/236 [01:47<00:00,  2.20it/s]


Epoch: 3                 | Train Loss:  0.004                 | Train Accuracy:  0.952                 | Val Loss:  0.017                 | Val Accuracy:  0.766
Early stopping counter 1/1
Early stopped, Best score :  0.8745519713261649
train finished


total_acc_train,▁▅█
total_acc_val,██▁
total_loss_train,█▄▁
total_loss_val,▁▁█
train_batch_loss,▄▁█
val_batch_loss,▁▁█
total_acc_train,7169
total_acc_val,641
total_loss_train,33.28057
total_loss_val,13.84475
train_batch_loss,0.44691


## 6. Test dataset으로 추론 (Prediction)


- v2 에서 수정된 부분
    - output -> output[0]

In [84]:
from torch.utils.data import DataLoader

# 테스트 데이터셋 불러오기
test_data = CustomDataset(test_df, tokenizer = TOKENIZER, max_len= args.max_seq_len, mode='test')

def test(model, SAVED_MODEL,test_data, args, mode = 'test'):


    test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=args.eval_batch_size)


    if use_cuda:

        model = model.to(DEVICE)
        model.load_state_dict(torch.load(SAVED_MODEL)['model'])


    model.eval()

    pred = []

    with torch.no_grad():
        for test_input in test_dataloader:

            mask = test_input['attention_mask'].to(DEVICE)
            input_id = test_input['input_ids'].squeeze(1).to(DEVICE)
            segment_ids = test_input['token_type_ids'].squeeze(1).to(DEVICE)

            output = model(input_id, mask, segment_ids)

            output = output[0].argmax(dim=1).cpu().tolist()

            for label in output:
                pred.append(label)
                
    return pred

SAVED_MODEL =  os.path.join(args.result_dir, f'best_{args.run}.pt')

pred = test(model,SAVED_MODEL, test_data, args)

In [85]:
print("prediction completed for ", len(pred), "comments")


prediction completed for  511 comments


### 

In [87]:
hate_dict = {0: 'none', 1: 'hate', 2: 'hate'} # hate speech 모델은 원래 none offensive hate 3가지로 분류하는 모델이었으나 이자료에서는 offensive와hate가 합쳐졌기 떄문에 둘다 hate로지정
pred_hate = ['' for i in range(len(pred))]

In [88]:
for idx, label in enumerate(pred):
    pred_hate[idx]=(str(hate_dict[label]))

In [89]:
submit = pd.read_csv(os.path.join(args.data_dir,'sample_submission.csv'))
submit

,ID,bias,hate
0,0,none,none
1,1,none,none
2,2,none,none
3,3,none,none
4,4,none,none
...,...,...,...
506,506,none,none
507,507,none,none
508,508,none,none
509,509,none,none


In [90]:
submit['hate'] = pred_hate

In [92]:
submit['hate'].value_counts()

hate    367
none    144
Name: hate, dtype: int64

In [158]:
submit.to_csv(os.path.join(args.result_dir, "submit_only_hate_normalized.csv"), index=False) # hate만 지정된 값을 일단 csv파일로만듬

In [203]:
submit_2 = pd.read_csv('./result/submit_only_bias_2.csv') #bias값만 지정해줬던 csv파일 불러옴

In [48]:
submit['bias'] = submit_2['bias']  # hate값만 있는 submit에 bias값만 있는 submit 파일을 합쳐줌

In [39]:
submit.to_csv(os.path.join(args.result_dir, "submit_last.csv"), index=False) #파일로저장